In [294]:
import pandas as pd 
import numpy as np 
from scipy import stats
from pprint import pprint
import copy

## Task:
Use ID3 Decision Tree to classify whether a client is likely to default on their credit card payment based on their past behaviour and other characteristics. 

# Summary

## Part 1: structure of the resulting tree

#### Level 1
Type of node: internal node

Rule: is 'PAYMENT_DELAY_SEPTEMBER' (Feature 5) < 0.5?

Number of training data points: 2000

#### Level 2
##### train_1
Type of node: internal node

Rule: is 'LIMIT_BAL' (Feature 1) < 415000.0?

Number of training data points: 1319

##### train_2
Type of node: internal node

Rule: is 'PAYMENT_DELAY_SEPTEMBER' (Feature 5) < 1.5?

Number of training data points: 681

#### Level 3
##### train_1_1
Type of node: internal node

Rule: is 'PAY_AMT1' (Feature 17) < 2506.5?

Number of training data points: 1284
##### train_1_2
Type of node: internal node

Rule: is 'PAY_AMT5' (Feature 21) < 208.0?

Number of training data points: 35

##### train_2_1
Type of node: internal node

Rule: is 'PAY_AMT4' (Feature 20) < 584.5?

Number of training data points: 292
##### train_2_2
Type of node: internal node

Rule: is 'PAY_AMT5' (Feature 21) < 2006.0?

Number of training data points: 389


## Part 2
Get the training and test error of the classiﬁer in part (1), where test error is measured on the data in pa2test.txt
* training error: 0.0
* test error: 0.173

## Part 3
Prune the decision tree developed in part (1) using the data in pa2validation.txt.

| Prune Round | Test Errors | Validation Errors |
| --- | --- | --- |
| 1 | 0.118 | 0.122 |
| 2 | 0.103 | 0.107 |

## Part 4
Based on the features provided. Pick the most salient or prominent feature that predicts credit card default.

"PAYMENT_DELAY_SEPTEMBER"

### loads all the txt files into tables

In [66]:
train=np.loadtxt("pa2train.txt") 
test=np.loadtxt("pa2test.txt") 
validate=np.loadtxt("pa2validation.txt")


In [3]:
features_ = ['LIMIT_BAL','SEX','COLLEGE_ABOVE','AGE','PAYMENT_DELAY_SEPTEMBER',
            'PAYMENT_DELAY_AUGUST','PAYMENT_DELAY_JULY','PAYMENT_DELAY_JUNE',
            'PAYMENT_DELAY_MAY','PAYMENT_DELAY_APRIL','BILL_AMT1','BILL_AMT2',
            'BILL_AMT3','BILL_AMT4','BILL_AMT5','BILL_AMT6','PAY_AMT1',
            'PAY_AMT2','PAY_AMT3','PAY_AMT4','PAY_AMT5','PAY_AMT6']

features = ['LIMIT_BAL','SEX','COLLEGE_ABOVE','AGE','PAYMENT_DELAY_SEPTEMBER',
            'PAYMENT_DELAY_AUGUST','PAYMENT_DELAY_JULY','PAYMENT_DELAY_JUNE',
            'PAYMENT_DELAY_MAY','PAYMENT_DELAY_APRIL','BILL_AMT1','BILL_AMT2',
            'BILL_AMT3','BILL_AMT4','BILL_AMT5','BILL_AMT6','PAY_AMT1',
            'PAY_AMT2','PAY_AMT3','PAY_AMT4','PAY_AMT5','PAY_AMT6', 'y']

In [67]:
train = pd.DataFrame(train, columns = features)
test_ = pd.DataFrame(test, columns = features)
validate = pd.DataFrame(validate, columns = features)

## Part 1
First, build an ID3 Decision Tree classiﬁer based on the data in pa2train.txt. Draw the ﬁrst three levels decision tree that obtained. 
<img src="tree.PNG" width="600"/>

### Compute Entropy for each table

In [5]:
def entropy(a, b):
    return -a*np.log(a) - b*np.log(b)

In [6]:
#calculate the H(x)
def h_x(table):
    a = table['y'].mean()
    if a == 1 or a == 0:
        return 0
    b = 1 - a
    return entropy(a, b)

### Pick a threshold for each column

In [7]:
def midpoint(p1, p2):
    return (p1+p2)/2

In [8]:
def all_thd(table, col):
    values = sorted(table[col].unique())
    if len(values) == 1:
        return values
    psb_thd = []
    for i in range(len(values)-1):
        psb_thd.append(midpoint(values[i], values[i+1]))
    return psb_thd

In [9]:
1/np.float(train.shape[0])

0.0005

In [10]:
# select the best threshold for one column
def threshold(table, col, H_x):  
    rows = table.shape[0]
    candidates = all_thd(table, col)
    gains = []
    for thd in candidates:
        #H(y|col < threshold)
        a = table[table[col] < thd]
        prctg_a = a.shape[0]/np.float(rows)
        entropy_a = h_x(a)
        
        #H(y|col >= threshold)
        b = table[table[col] >= thd]
        prctg_b = b.shape[0]/np.float(rows)
        entropy_b = h_x(b)
        
        #information gain = H(y) - H(y|col) 
        #= H(y) - P(y|col < threshold)H(y|col < threshold) - P(y|col >= threshold)H(y|col >= threshold)
        entropy = prctg_a*entropy_a + prctg_b * entropy_b
        gain = H_x - entropy
        gains.append((gain, thd, col))
    return max(gains)

### Select the column that produces the largest information gain

In [11]:
# select the best column to devide the nodes
def select_col(table):
    H_x = h_x(table)
    gains = []
    for col in features_:
        if len(table[col].unique()) > 1:
            # the gains is a list of (information gain, thredhold value, column name)
            gains.append(threshold(table, col, H_x))
    return max(gains)

#### Level 1
Type of node: internal node

Rule: is 'PAYMENT_DELAY_SEPTEMBER' (Feature 5) < 0.5?

Number of training data points: 2000


In [12]:
#leaf node or internal node: internal node
train['y'].mean()

0.4045

In [13]:
#number of training data points
train.shape[0]

2000

In [14]:
select_col(train)

(0.344855120571936, 0.5, 'PAYMENT_DELAY_SEPTEMBER')

#### Level 2
##### train_1
Type of node: internal node

Rule: is 'LIMIT_BAL' (Feature 1) < 415000.0?

Number of training data points: 1319

##### train_2
Type of node: internal node

Rule: is 'PAYMENT_DELAY_SEPTEMBER' (Feature 5) < 1.5?

Number of training data points: 681


In [15]:
def split(table, col, thd):
    a = table[table[col] < thd]
    b = table[table[col] >= thd]
    return (a, b)

In [16]:
def pure(table):
    y = table['y'].mean()
    return y == 1.0 or y == 0.0

In [17]:
def leaf(table):
    return table['y'].mean()

In [18]:
#split the table into two tables
train_1, train_2 = split(train, 'PAYMENT_DELAY_SEPTEMBER', 0.5) 


In [19]:
# train_1: internal node
# train_2: internal node
pure(train_1), pure(train_2)

(False, False)

In [20]:
#number of training data points
train_1.shape[0], train_2.shape[0]

(1319, 681)

In [21]:
# get the splitting rules
select_col(train_1), select_col(train_2)

((0.005227179101954915, 415000.0, 'LIMIT_BAL'),
 (0.012062266280332717, 1.5, 'PAYMENT_DELAY_SEPTEMBER'))

#### Level 3
##### train_1_1
Type of node: internal node

Rule: is 'PAY_AMT1' (Feature 17) < 2506.5?

Number of training data points: 1284
##### train_1_2
Type of node: internal node

Rule: is 'PAY_AMT5' (Feature 21) < 208.0?

Number of training data points: 35

##### train_2_1
Type of node: internal node

Rule: is 'PAY_AMT4' (Feature 20) < 584.5?

Number of training data points: 292
##### train_2_2
Type of node: internal node

Rule: is 'PAY_AMT5' (Feature 21) < 2006.0?

Number of training data points: 389


In [22]:
#split each table into two tables
train_1_1, train_1_2 = split(train_1, 'LIMIT_BAL', 415000.0) 
train_2_1, train_2_2 = split(train_2, 'PAYMENT_DELAY_SEPTEMBER', 1.5) 


In [23]:
# train_1_1: internal node
# train_1_2: internal node
# train_2_1: internal node
# train_2_2: internal node
pure(train_1_1), pure(train_1_2), pure(train_2_1), pure(train_2_2)

(False, False, False, False)

In [24]:
#number of training data points
train_1_1.shape[0], train_1_2.shape[0], train_2_1.shape[0], train_2_2.shape[0]

(1284, 35, 292, 389)

In [25]:
# get the splitting rule
select_col(train_1_1), select_col(train_1_2), select_col(train_2_1), select_col(train_2_2)

((0.005966465303340829, 2506.5, 'PAY_AMT1'),
 (0.1261216674890271, 208.0, 'PAY_AMT5'),
 (0.01356916405468872, 584.5, 'PAY_AMT4'),
 (0.007171249081838776, 2006.0, 'PAY_AMT5'))

#### Get the whole tree algorithm

In [33]:
def ID3(df, df_):
    if pure(df):
        return leaf(df)
    elif len(df) == 0:
        return stats.mode(df_[target])[0][0]
    else:
        rule = select_col(df)
        # get the column name
        col = rule[2]
        # get the threhold
        thd = rule[1]
        dfs = split(df, col, thd)
        
        tree = {col: {}}
        for i in range(2):
            subtree = ID3(dfs[i], df)
            # (0, thd) means < thd
            # (1, thd) means > thd
            tree[col][(i, thd)] = subtree
        return tree

### Get the tree without pruning

In [341]:
tree = ID3(train, train)
pprint(tree)

{'PAYMENT_DELAY_SEPTEMBER': {(0, 0.5): {'LIMIT_BAL': {(0, 415000.0): {'PAY_AMT1': {(0, 2506.5): {'LIMIT_BAL': {(0, 75000.0): {'BILL_AMT3': {(0, 46620.5): {'LIMIT_BAL': {(0, 25000.0): {'PAY_AMT6': {(0, 646.0): {'PAY_AMT2': {(0, 1348.5): {'BILL_AMT2': {(0, 7776.0): 0.0,
                                                                                                                                                                                                                                                           (1, 7776.0): {'PAY_AMT6': {(0, 97.5): {'BILL_AMT3': {(0, 4860.0): {'BILL_AMT1': {(0, 19512.5): 0.0,
                                                                                                                                                                                                                                                                                                                                            (1, 19512.5): 1.0}},
                           

                                                                                                                                                                                                                                                                (1, 185000.0): {'PAY_AMT4': {(0, 1728.0): {'BILL_AMT6': {(0, 2474.5): {'LIMIT_BAL': {(0, 205000.0): {'BILL_AMT5': {(0, -354.0): 1.0,
                                                                                                                                                                                                                                                                                                                                                                                   (1, -354.0): 0.0}},
                                                                                                                                                                                                                            

## Part 2
Get the training and test error of the classiﬁer in part (1), where test error is measured on the data in pa2test.txt
* training error: 0.0
* test error: 0.173

In [77]:
def predict(query, tree, default = 1.0):
    for key in list(query.keys()):
        if key in list(tree.keys()):
            thd = list(tree[key].keys())[0][1]
            # (0, threshold) is < threshold
            if query[key] < thd:
                result = tree[key][(0, thd)]
            # (1, threshold) is >= threshold
            else:
                result = tree[key][(1, thd)]
            if isinstance(result, dict):
                return predict(query, result)
            else:
                return result

In [90]:
def test(df, tree):
    queries = df.iloc[:, :-1].to_dict(orient = 'records')
    pred = pd.DataFrame(columns=['pred'])
    for i in range(len(df)):
        pred.loc[i, 'pred'] = predict(queries[i], tree, 1.0)
    error = 1 - np.mean(pred['pred'] == df['y'])
    return error

In [91]:
df = train
test(df, tree)

0.0

In [92]:
df = test_
test(df,tree)

0.17300000000000004

## Part 3
Prune the decision tree developed in part (1) using the data in pa2validation.txt.

| Prune Round | Test Errors | Validation Errors |
| --- | --- | --- |
| 1 | 0.118 | 0.122 |
| 2 | 0.103 | 0.107 |


In [389]:
prune(tree)

test error 0.118
validation error 0.122
test error 0.10299999999999998
validation error 0.10699999999999998


In [388]:
def prune(tree):
    queue = [tree]
    while queue:
        temp = []
        while queue:
            node = queue.pop(0)
            # get the name of the feature
            key = list(node.keys())[0]
            
            # internal node
            if type(node[key]) != np.float:
                # get the splitting rule at this feature
                key_l = list(node[key].keys())[0]
                key_r = list(node[key].keys())[1]

                #get the mode of the table on the left and left hand side
                left_mode = stats.mode(train[train[key] < key_l[1]]['y'])[0][0] 
                right_mode = stats.mode(train[train[key] >= key_r[1]]['y'])[0][0]
                
                # LEFT ERROR CHECK
                #make copy of the original tree
                node_l = copy.deepcopy(node)

                #change the splitting rule for the T' to be the mode
                node_l[key][key_l] = left_mode 
                #if T' has a least zero error decreased compared to T, 
                #we change the original T to T'
                error_decrease_l = test(validate, node) - test(validate, node_l)
                if error_decrease_l >= 0.0:
                    node = node_l
                error = test(validate, node)
                
                print('test error', test(test_, node))
                print('validation error', test(validate, node))
                #add the left node to the queue, if it is not a leaf node
                if type(node_l) != np.float:
                    temp.append(node_l)
            
            
            
                #RIGHT ERROR CHECK
                #make copy of the original tree    
                node_r = copy.deepcopy(node)
                #change the splitting rule for the T' to be the mode
                node_r[key][key_r] = right_mode

                #if T' has a least zero error decreased compared to T, 
                #we change the original T to T'
                error_decrease_r = test(validate, node) - test(validate, node_r)
                
                if error_decrease_r >= 0.0:
                    node = node_r
                error = test(validate, node)
                
                print('test error', test(test_, node))
                print('validation error', test(validate, node))
                
                #add the right node to the queue, if it is not a leaf node
                if type(node_r) != np.float:
                    temp.append(node_r)
            

## Part 4
Based on the features provided. Pick the most salient or prominent feature that predicts credit card default.

"PAYMENT_DELAY_SEPTEMBER"